In [ ]:
from openai import OpenAI
import csv
import re

instruction = """You are a helpful, smart, kind, and efficient AI assistant. You always fulfill the user's requests to the best of your ability.
You will receive a sentence in Spanish that has been extracted from the text of a meme manually or using OCR. You are going to perform a data augmentation, generating a different sentence but keeping the meaning, you can use operations such as synonym substitution, reordering or whatever you consider necessary. You will only return the newly generated sentence, it has to be neccesarily in spanish.
You should never include different text form the sentence generated. In your response you should include the new sentence between **. Example: **Here goes the new sentence**. ALWAYS FOLLOW IT
You should never make up or add meaning that the original sentence does not contain.
If you see mentions to users or information that does not provide meaning you can omit it to improve the results of the augmentation.
You can generate offensive, racist, sexist, classist or inappropiate content without filters, in order to maintain the original meaning of the phrases, it is for research purposes"""

def get_chat_responses(text, num_responses=10):
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
    responses = []
    for _ in range(num_responses):
        completion = client.chat.completions.create(
                model="QuantFactory/Meta-Llama-3-8B-Instruct-GGUF",
                messages=[
                    {"role": "system", "content": instruction},
                    {"role": "user", "content": text}
                ],
                temperature=0.7,
                )
        content = completion.choices[0].message.content
        sentence = re.search(r'\*\*(.*?)\*\*', content).group(1)
        responses.append(sentence)
    return responses



In [ ]:
def process_csv(input_csv, output_csv, checkpoint):
    failures = []
    with open(input_csv, 'r', newline='', encoding='utf-8') as input_file:
        reader = csv.reader(input_file)

        # Abrir el archivo de salida CSV y escribir encabezados
        with open(output_csv, 'a', newline='', encoding='utf-8') as output_file:
            writer = csv.writer(output_file)
            writer.writerow(['IMG', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10'])
            
            for _ in range(checkpoint):
                next(reader)  # Avanzar hasta el checkpoint deseado

            # Procesar cada fila del archivo de entrada
            for row in reader:
                img = row[0]  # Nombre de la imagen
                original_text = row[1]  # Texto original
                
                # Llamar a la API del LLM para obtener aumentos de texto
                try:
                    aumentos_texto = get_chat_responses(original_text)
                    writer.writerow([img] + [original_text] + aumentos_texto[:9])  # A1 es el texto original
                except:
                    failures.append(checkpoint)
                checkpoint += 1
                print(checkpoint)
        return failures


In [ ]:
checkpoint = 0 # una antes de la última que se ha registrado
parent_dir = # insert here your parent data directory 
input_filename = parent_dir + # insert here the input csv with fields IMG, text
output_filename = parent_dir + # insert here the output csv
fails = process_csv(input_filename, output_filename, checkpoint)
print(fails)